# Series Module

Copyright 2022 Michael George (AKA Logiqx).

This file is part of [sse-results](https://github.com/Logiqx/sse-results) and is distributed under the terms of the GNU General Public License.

sse-results is free software: you can redistribute it and/or modify it under the terms of the GNU General Public License as published by the Free Software Foundation, either version 3 of the License, or (at your option) any later version.

sse-results is distributed in the hope that it will be useful, but WITHOUT ANY WARRANTY; without even the implied warranty of MERCHANTABILITY or FITNESS FOR A PARTICULAR PURPOSE. See the GNU General Public License for more details.

You should have received a copy of the GNU General Public License along with sse-results. If not, see <https://www.gnu.org/licenses/>.

## Initialisation

Basic approach to determine the project directory

In [1]:
import os
import glob

from datetime import datetime
import time

import json
import jinja2

from PIL import Image

from common import Printable, projdir
from event import Event
from constants import *

## Leg Class

Process leg held during a single session

In [2]:
class Leg(Printable):
    def __init__(self, series, session, legId):
        '''Initialise leg object'''

        self.series = series
        self.session = session
        self.legId = legId


    def determineValidity(self):
        '''Determine if the lef if valid'''

        courses = set()

        self.participants = 0

        for entrantId, runs in self.session.runs.items():
            if entrantId in self.series.entrants:
                self.participants += 1

                for run in runs:
                    course = run.course.courseType
                    if course not in courses:
                        courses.add(course)

        print(self.session.date, self.legId, len(self.series.entrants), self.participants, courses)


    def processLeg(self):
        '''Process leg held during the session'''

        if self.series.validity:
            pass           
        else:
            self.determineValidity()

## Series Class

Process series held during a single event

In [3]:
class Series(Printable):
    def __init__(self, event, seriesId, fleetId=None, validity=None):
        '''Initialise series object'''

        self.event = event
        self.seriesId = seriesId
        self.fleetId = fleetId
        self.validity = validity

        self.seriesConfig = self.event.eventConfig['Series'][seriesId]

        seriesFilter = self.seriesConfig['Filter']

        # fleetID == None for a "verification" series
        if fleetId:
            fleetFilter = self.seriesConfig['Fleets'][self.fleetId]
        else:
            fleetFilter = self.seriesConfig['Validity']['Filter']

        # Verification series may not have an additional filter criteria
        if fleetFilter:
            filterCriteria = ','.join((seriesFilter, fleetFilter))
        else:
            filterCriteria = seriesFilter

        self.entrants = self.event.entrantsFilter.getEntrants(filterCriteria)
        
        self.legs = []


    def processSeries(self):
        '''Process series held during the event'''

        print(self.event.year, self.seriesId, self.fleetId, len(self.entrants))

        if self.validity is None:
            # Validity leg will read the config files
            for sessionId, session in self.event.sessions.items():
                filename = os.path.join(session.configPath, self.seriesId.lower() + '.json')
                with open(filename, 'r', encoding='utf-8') as f:
                    jsonTxt = f.read()
                    try:
                        legsConfig = json.loads(jsonTxt)
                    except:
                        self.logError('Could not parse {}'.format(filename))
                        raise

                for legId in legsConfig.keys():
                    leg = Leg(self, session, legId)
                    leg.processLeg()

                    self.legs.append(leg)

        else:
            # Actual leg will use the validity leg as a template
            for validityLeg in self.validity.legs:
                leg = Leg(self, validityLeg.session, validityLeg.legId)
                leg.processLeg()

                self.legs.append(leg)

## Process Years

Process all available years

In [4]:
events = {}
existingNames = {}

def processEvents():
    '''Process all events from 2010 onwards'''

    eventPaths = sorted(glob.glob(os.path.join(projdir, EVENTS_DIR, '[2][0-9][1][0]')))
    currYear = currYear = os.path.basename(eventPaths[-1])
    for eventPath in eventPaths:
        eventYear = os.path.basename(eventPath)

        if eventYear == currYear or appConfig['History']['Refresh']:
            if eventYear == currYear:
                verbosity = appConfig['Latest']['Verbosity']
            else:
                verbosity = appConfig['History']['Verbosity']

            event = Event(eventPath, appConfig, existingNames=existingNames, verbosity=verbosity)
            event.processEvent(runReports=False)
            
            if 'Series' in event.eventConfig:
                for seriesId in event.eventConfig['Series'].keys():
                    validity = Series(event, seriesId)
                    validity.processSeries()

                    for fleetId in event.eventConfig['Series'][seriesId]['Fleets'].keys():
                        series = Series(event, seriesId, fleetId=fleetId, validity=validity)
                        series.processSeries()

            events[event.year] = event

In [5]:
if __name__ == '__main__':
    pc1 = time.perf_counter()
    
    # Read main config
    filename = os.path.join(projdir, CONFIG_DIR, APP_CONFIG)
    with open(filename, 'r', encoding='utf-8') as f:
        jsonTxt = f.read()
        try:
            appConfig = json.loads(jsonTxt)
        except:
            logger = Printable()
            logger.logError('Could not parse {}'.format(filename))
            raise
    
    processEvents()
    
    pc2 = time.perf_counter()
    print("Reports completed in %0.2f seconds" % (pc2 - pc1))

Processing 2010...
INFO: 270 runs by 42 participants down course H1 on 20101016 - 11:51:23 to 15:27:25
INFO: 27 runs by 9 participants down course H1 on 20101017 - 10:34:09 to 12:06:37
INFO: 13 runs by 2 participants down course H1 on 20101018 - 11:59:49 to 13:53:16
INFO: 205 runs by 38 participants down course S1 on 20101018 - 11:38:06 to 13:28:59
INFO: 283 runs by 49 participants down course S2 on 20101018 - 13:30:25 to 16:29:57
INFO: 5 runs by 2 participants down course H1 on 20101019 - 15:02:09 to 16:14:54
INFO: 221 runs by 42 participants down course S1 on 20101019 - 11:17:01 to 12:59:54
INFO: 673 runs by 69 participants down course S2 on 20101019 - 14:04:46 to 16:59:55
INFO: 22 duplicate runs removed for Patrick Van Hoof (Sailboard) on 20101019 - 50% dupes
INFO: 4 runs by 2 participants down course S3 on 20101019 - 12:37:25 to 15:24:08
INFO: 25 runs by 7 participants down course H1 on 20101020 - 11:06:23 to 12:55:32
INFO: 5 runs by 1 participants down course H2 on 20101020 - 14:3

## All Done!